# Deep Neural Network

Source: https://github.com/haradai1262/CIKM2020-AnalytiCup

TO DO
- faire le dnn avec pytorch


In [3]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sns as sns
from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit  # pip install verstack


# Load the training data
train_data = pd.read_csv("data/train.csv")
train_data=train_data.drop(['mentions'], axis=1)

# Load the evaluation data
eval_data = pd.read_csv("data/evaluation.csv")
eval_data=eval_data.drop(['mentions'], axis=1)

# split data
X_train, X_test, y_train, y_test = scsplit(train_data, train_data['retweets_count'], stratify=train_data['retweets_count'], train_size=0.8, test_size=0.2)
# We remove the actual number of retweets from our features since it is the value that we are trying to predict
X_train = X_train.drop(['retweets_count'], axis=1)
X_test = X_test.drop(['retweets_count'], axis=1)

c:\Users\maria\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Extrac features


## Extractions

## time


In [4]:
from datetime import datetime
def extract_time_features(df):
    rs_df = df
    rs_df["hour"] = rs_df['timestamp'].apply(
        lambda t: (datetime.fromtimestamp(t//1000).hour))
    rs_df["day"] = rs_df['timestamp'].apply(
        lambda t: (datetime.fromtimestamp(t//1000)).weekday())
    rs_df["week_in_month"] = rs_df['timestamp'].apply(
        lambda t: (datetime.fromtimestamp(t//1000).day)//7)  
    rs_df=rs_df.drop(['timestamp'], axis=1)
    return rs_df

## Num

In [5]:

def extract_ratio_features(df):
    rs_df = df
    rs_df['followers__favorites'] = rs_df['followers_count'] * rs_df['favorites_count']
    rs_df['friends__favorites'] = rs_df['friends_count'] * rs_df['favorites_count']
    rs_df['followers__friends__favorites'] = rs_df['followers_count'] * rs_df['friends_count'] * rs_df['favorites_count']
    return rs_df

from scipy.stats import norm
from scipy.stats import zscore
def extract_transfo(df,columns):
    rs_df = df
    for col in columns:
        mean = rs_df[col].mean()
        std = rs_df[col].std()
        rs_df[col+'_cdf'] = norm.cdf(rs_df[col].values, loc=mean, scale=std)
        rs_df[col+'_z'] = zscore(rs_df[col].values)       
        rs_df[col+'_rank'] = rs_df[col].rank(method='min')
        rs_df[col+'log'] = (df[col] + 1).apply(np.log)
    return rs_df
    

## Text

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
def extract_topic(df):
    rs_df = df
    rs_df['hashtags'] = rs_df['hashtags'].apply(
        lambda x: x.replace('[', '').replace(']', '').replace("'", ''))
    #join text and hashtags
    rs_df['total_text'] = rs_df['text'] + ' ' + rs_df['hashtags']
    vectorizer = TfidfVectorizer(min_df=1, max_features=None, stop_words=stopwords.words('french'))
    vector = vectorizer.fit_transform(rs_df['text'])
    svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
    svd.fit(vector)
    topic=svd.transform(vector)
    rs_df['topic_1'] = topic[:,0]
    rs_df['topic_2'] = topic[:,1]
    rs_df['topic_3'] = topic[:,2]
    rs_df['topic_4'] = topic[:,3]
    rs_df['topic_5'] = topic[:,4]
    rs_df=rs_df.drop(['hashtags'],axis=1)
    rs_df=rs_df.drop(['total_text'],axis=1)
    return rs_df

from textblob import TextBlob  # pip install textblob-fr
from textblob_fr import PatternTagger, PatternAnalyzer


def sent_engineering(in_df):
    rs_df = in_df
    # add columns related to sentiment analysis
    rs_df['polarity'] = rs_df['text'].apply(lambda x: TextBlob(
        x, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer()).sentiment[0])
    rs_df['subjectivity'] = rs_df['text'].apply(lambda x: TextBlob(
        x, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer()).sentiment[1])
    # drop the text column
    rs_df = rs_df.drop(['text'], axis=1)
    return rs_df

def extract_url(in_df):
    #count url
    rs_df = in_df
    rs_df['url_count'] = rs_df['urls'].apply(lambda x: len(x.split(',')))
    rs_df=rs_df.drop(['urls'],axis=1)

    return rs_df


In [11]:
#import kmeans
from sklearn.cluster import KMeans
def extract_cluster(df,columns):
    rs_df = df
    rs_df['cluster'] = KMeans(n_clusters=100, random_state=0).fit_predict(rs_df[columns].values)
    return rs_df

In [8]:
tweet_metrics_features = [
    'followers_count', 'friends_count', 'favorites_count',
    'followers__favorites', 'friends__favorites', 'followers__friends__favorites',
]

tweet_metrics_log_features = [feat+'_log' for feat in tweet_metrics_features]
tweet_metrics_cdf_features = [feat+'_cdf' for feat in tweet_metrics_features]
tweet_metrics_z_features = [feat+'_z' for feat in tweet_metrics_features]
tweet_metrics_rank_features = [feat+'_rank' for feat in tweet_metrics_features]

time_cat_features = ['hour', 'day', 'week_of_month']

In [12]:


new_X_train=extract_time_features(X_train)
new_X_train=extract_ratio_features(new_X_train)
new_X_train=extract_transfo(new_X_train,tweet_metrics_features)
new_X_train=extract_topic(new_X_train)
new_X_train=sent_engineering(new_X_train)
new_X_train=extract_url(new_X_train)
new_X_train=extract_cluster(new_X_train,['followers_count', 'friends_count', 'favorites_count','verified','statuses_count'])




            

        

In [ ]:
print(new_X_train.head())


        favorites_count  followers_count  statuses_count  friends_count  \
145711                0              528           32283            236   
174690                1              454             216            559   
163477                0               29            1133            315   
202410                0              679           28413            759   
219776                0              412              73            608   

        verified  TweetID  hour  day  week_in_month  followers__favorites  \
145711         0   698089    22    3              1                     0   
174690         0   167320    17    5              1                   454   
163477         0   221247    13    5              1                     0   
202410         0   607469    20    3              1                     0   
219776         0   264674    16    5              1                     0   

        ...  followers__friends__favorites_rank  \
145711  ...                        

In [15]:
new_X_test=extract_time_features(X_test)
new_X_test=extract_ratio_features(new_X_test)
new_X_test=extract_transfo(new_X_test,tweet_metrics_features)
new_X_test=extract_topic(new_X_test)
new_X_test=sent_engineering(new_X_test)
new_X_test=extract_url(new_X_test)
new_X_test=extract_cluster(new_X_test,['followers_count', 'friends_count', 'favorites_count','verified','statuses_count'])

In [19]:
#import mlp
from sklearn.neural_network import MLPRegressor
remove=['TweetID']
model=MLPRegressor(hidden_layer_sizes=(2048,512,128),activation='relu',
                    solver='adam',alpha=0.001,batch_size='auto')
model.fit(new_X_train.drop(remove, axis=1),y_train)
y_pred = model.predict(new_X_test.drop(remove, axis=1))
print(mean_absolute_error(y_test, y_pred))

# DNN with Pytorch

In [ ]:
#Buid an MLP with pytoch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(44, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 1)
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.2)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = self.dropout1(out)

        out = F.relu(self.fc2(out))
        out = self.dropout2(out)

        out = F.relu(self.fc3(out))
        out = self.dropout3(out)

        out = self.fc4(out)

        return out

#convert to tensor
X_train_tensor=torch.tensor(new_X_train.drop(remove, axis=1).values)
X_test_tensor=torch.tensor(new_X_test.drop(remove, axis=1).values)
y_train_tensor=torch.tensor(y_train.values)
y_test_tensor=torch.tensor(y_test.values)




In [ ]:
# Instantiate model class and assign to object
model_dropout= DNN()

# Optimizer
optimizer = torch.optim.RMSprop(model_dropout.parameters(), lr=1e-3)

# Loss function
criterion = torch.nn.L1Loss()

(train_acc, val_acc, train_loss, val_loss) = train_model(model_dropout, optimizer, criterion)

def plot_losses(history_train_loss, history_val_loss):
    # Set plotting style
    #plt.style.use(('dark_background', 'bmh'))
    plt.style.use('bmh')
    plt.rc('axes', facecolor='none')
    plt.rc('figure', figsize=(16, 4))

    # Plotting loss graph
    plt.plot(history_train_loss, label='Train')
    plt.plot(history_val_loss, label='Validation')
    plt.title('Loss Graph')
    plt.legend()
    plt.show()
    
plot_losses(train_loss, val_loss)
